In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys
import boto3
import yaml

current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.insert(0, root_dir)

from scripts.common_functions import *

In [2]:
access_key, access_secret = get_aws_creds()

spark = create_spark_session_with_s3('practice', access_key, access_secret)

In [3]:
# s3_client = boto3.client('s3')
# response = s3_client.list_objects_v2(Bucket='cricket-bucket-obi')
# for item in response.get('Contents', []):
#     print(item['Key'])

In [ ]:
s3_path = 's3://cricket-bucket-obi/raw_cricsheet_data/1001349.json'
df = spark.read.option('multiline', True).json(os.path.join(s3_path))
df.show()

Py4JJavaError: An error occurred while calling o45.json.
: java.io.IOException: From option fs.s3a.aws.credentials.provider java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.auth.IAMInstanceCredentialsProvider not found
	at org.apache.hadoop.fs.s3a.S3AUtils.loadAWSProviderClasses(S3AUtils.java:631)
	at org.apache.hadoop.fs.s3a.S3AUtils.createAWSCredentialProviderSet(S3AUtils.java:597)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:257)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(Data

In [ ]:
spark.stop()